In [ ]:
#Main code for 0.5 Million trainable parameter
import pathlib
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from sklearn.metrics import confusion_matrix

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set paths and parameters
partition = "Part2"
base_path = pathlib.Path("/content/drive/My Drive/FYP")  # Update if needed

train_path = base_path / partition / "Train"
test_path = base_path / partition / "Test"
results_path = base_path / partition / "results"
output_path = base_path / partition / "output"

# Ensure results and output directories exist
for path in [results_path, output_path]:
    os.makedirs(path, exist_ok=True)

# Verify paths
print(f"Train Path: {train_path}")
print(f"Test Path: {test_path}")
print(f"Results Path: {results_path}")
print(f"Output Path: {output_path}")

# Data initialization
datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.1, rescale=1.0 / 255)

train_dataset = datagen.flow_from_directory(
    train_path, target_size=(120, 160), class_mode="binary", seed=0, subset="training"
)
validation_dataset = datagen.flow_from_directory(
    train_path, target_size=(120, 160), class_mode="binary", seed=0, subset="validation"
)
test_dataset = datagen.flow_from_directory(
    test_path, target_size=(120, 160), class_mode="binary", seed=0
)

# Model architecture
batchX, _ = next(train_dataset)
input_shape = [batchX.shape[1], batchX.shape[2], batchX.shape[3]]

model = tf.keras.Sequential([
    # Reshape input if needed
    tf.keras.layers.Reshape(target_shape=(input_shape[0], input_shape[1] * input_shape[2])),  # Reshape to (height, width*channels)

    # First GRU layer for sequence processing
    tf.keras.layers.GRU(64, return_sequences=True),  # Set return_sequences=True to allow stacking more GRU layers

    # Second GRU layer
    tf.keras.layers.GRU(128, return_sequences=True),  # Increase the number of units for more complex representation

    # Third GRU layer
    tf.keras.layers.GRU(256, return_sequences=False),  # Further increase the number of units

    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.7),  # Increase dropout to prevent overfitting

    # Dense layers for classification
    tf.keras.layers.Dense(128, activation="relu"),  # Add a larger Dense layer
    tf.keras.layers.Dense(64, activation="relu"),   # Add another Dense layer
    tf.keras.layers.Dense(1, activation="sigmoid")  # Final output layer
])

# Manually build the model with the input shape
model.build(input_shape=(None, batchX.shape[1], batchX.shape[2], batchX.shape[3]))

print(f"Total number of trainable parameters: {model.count_params()}")

# Custom callback for plotting accuracy
class AccuracyPlotCallback(tf.keras.callbacks.Callback):
    def __init__(self, output_path):
        super().__init__()
        self.output_path = output_path
        self.epoch_count = []
        self.train_accuracy = []
        self.val_accuracy = []

    def on_epoch_end(self, epoch, logs=None):
        self.epoch_count.append(epoch + 1)
        self.train_accuracy.append(logs.get('accuracy'))
        self.val_accuracy.append(logs.get('val_accuracy'))

    def on_train_end(self, logs=None):
        plt.figure()
        plt.plot(self.epoch_count, self.train_accuracy, label='Training Accuracy')
        plt.plot(self.epoch_count, self.val_accuracy, label='Validation Accuracy')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(loc='best')
        plt.savefig(f"{self.output_path}/accuracy_plot_final.png")
        plt.close()

# Model compilation
model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002),
    metrics=["accuracy"]
)

# Callbacks
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=str(results_path / "saee.weights.h5"),
    save_weights_only=True,
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_freq="epoch"
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=40,
    restore_best_weights=True
)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=str(results_path / "log"), histogram_freq=1)
accuracy_plot_callback = AccuracyPlotCallback(output_path=str(output_path))

# Training the model
train_history = model.fit(
    x=train_dataset,
    batch_size=16,
    epochs=100,
    steps_per_epoch=len(train_dataset) // 16,
    verbose=1,
    validation_data=validation_dataset,
    callbacks=[
        model_checkpoint_callback,
        early_stopping_callback,
        tensorboard_callback,
        accuracy_plot_callback
    ]
)

# Save the model
model_file = str(results_path / "model_MimicPerformAF_100e.keras")
model.save(model_file)

# Model evaluation
scores = model.evaluate(test_dataset, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
model.summary()

# Confusion matrix
def calculate_confusion_matrix(dataset, dataset_name):
    y_pred = model.predict(dataset).round()
    y_true = dataset.classes
    conf_m = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(pd.DataFrame(conf_m), annot=True, fmt="g", cmap="Blues")
    plt.title(f"{dataset_name} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.savefig(f"{results_path}/{dataset_name}_confusion_matrix.png")
    plt.close()

calculate_confusion_matrix(test_dataset, "test")
calculate_confusion_matrix(validation_dataset, "validation")

# Accuracy and loss plots
def plot_metrics(history, metric, output_path):
    plt.figure()
    plt.plot(history.history[metric], label=f'Train {metric.capitalize()}')
    plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}')
    plt.title(f'Training and Validation {metric.capitalize()}')
    plt.xlabel('Epochs')
    plt.legend()
    plt.savefig(f"{output_path}/{metric}_plot.png")
    plt.close()

plot_metrics(train_history, 'accuracy', str(output_path))
plot_metrics(train_history, 'loss', str(output_path))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train Path: /content/drive/My Drive/FYP/Part2/Train
Test Path: /content/drive/My Drive/FYP/Part2/Test
Results Path: /content/drive/My Drive/FYP/Part2/results
Output Path: /content/drive/My Drive/FYP/Part2/output
Found 1512 images belonging to 2 classes.
Found 168 images belonging to 2 classes.
Found 420 images belonging to 2 classes.
Total number of trainable parameters: 518017
Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 55s 20s/step - accuracy: 0.6667 - loss: 0.6635 - val_accuracy: 0.4643 - val_loss: 0.6945
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 18s 9s/step - accuracy: 0.9479 - loss: 0.5722 - val_accuracy: 0.5357 - val_loss: 0.6867
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 4s/step - accuracy: 0.9740 - loss: 0.4632 - val_accuracy: 0.5357 - val_loss: 0.6812
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 16s 7s/step - accuracy: 1.0000 - loss: 0.3402 - val_accuracy: 0.5357 - val_los

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_1 (Reshape)             │ (None, 120, 480)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 120, 64)        │       104,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ (None, 120, 128)       │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 256)            │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,553,029 (5.92 MB)

 Trainable params: 517,505 (1.97 MB)

 Non-trainable params: 512 (2.00 KB)

 Optimizer params: 1,035,012 (3.95 MB)

14/14 ━━━━━━━━━━━━━━━━━━━━ 12s 765ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 658ms/step
